In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.preprocessing import MultiLabelBinarizer
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from tensorflow.keras.applications import InceptionResNetV2
import urllib.request

pd.set_option("display.max_columns", None)

In [44]:
train = pd.read_csv('train.csv', header=None, names=["image", "labels"])
print("Train ammount:",len(train))
print("Train columns:", train.columns)
print(train['labels'].value_counts())

Train ammount: 64
Train columns: Index(['image', 'labels'], dtype='object')
TwistedFate    1
Garen          1
Ahri           1
Orianna        1
Tryndamere     1
              ..
Fiora          1
Tristana       1
Diana          1
DrMundo        1
Kaisa          1
Name: labels, Length: 64, dtype: int64


In [45]:
# %%time
# datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
#                                                         preprocessing_function=None,
#                                                         data_format=None,
#                                                     )

# train_data = datagen.flow_from_dataframe(
#     train,
#     directory='./train_dataset',
# #     directory='./train_dataset_round',
#     x_col="image",
#     y_col= 'labels',
#     color_mode="rgb",
#     target_size = (256,256),
#     class_mode="categorical",
#     batch_size=32,
#     shuffle=False,
#     seed=40,
# )

datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_data = datagen.flow_from_dataframe(
    train,
    directory='./train_dataset',
#     directory='./train_dataset_round',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (299,299),
    class_mode="categorical",
    batch_size=16,
    shuffle=False,
    seed=40,
)

Found 64 validated image filenames belonging to 64 classes.


In [46]:
seed = 1200
tf.random.set_seed(seed)
num_classes = 64

# url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

# weights_path = './keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'

url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.7/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'

weights_path = './keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5'

if not os.path.exists(weights_path):
    urllib.request.urlretrieve(url, weights_path)
    
# Load the pre-trained InceptionResNetV2 model
base_model = keras.applications.InceptionResNetV2(weights=weights_path, include_top=True, input_shape=(299, 299, 3))

# Remove the last layer of the model
x = base_model.layers[-2].output

# Add a new fully connected layer with 64 output neurons
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
output = keras.layers.Dense(num_classes, activation='softmax')(x)

# Create a new model with the modified architecture
new_model = keras.models.Model(inputs=base_model.input, outputs=output)

print(new_model.input)
print(new_model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_13'), name='input_13', description="created by layer 'input_13'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='dense_9/Softmax:0', description="created by layer 'dense_9'")


In [47]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-5),
                  metrics=['accuracy'])

# Freezing the weights
for layer in base_model.layers[-20:]:
    layer.trainable = True
    
new_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2436 (Conv2D)           (None, 149, 149, 32  864         ['input_13[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_2436 (Batc  (None, 149, 149, 32  96         ['conv2d_2436[0][0]']            
 hNormalization)                )                                                           

                                                                                                  
 conv2d_2443 (Conv2D)           (None, 35, 35, 64)   76800       ['activation_2442[0][0]']        
                                                                                                  
 conv2d_2446 (Conv2D)           (None, 35, 35, 96)   82944       ['activation_2445[0][0]']        
                                                                                                  
 conv2d_2447 (Conv2D)           (None, 35, 35, 64)   12288       ['average_pooling2d_12[0][0]']   
                                                                                                  
 batch_normalization_2441 (Batc  (None, 35, 35, 96)  288         ['conv2d_2441[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 batch_nor

                                                                                                  
 block35_1_mixed (Concatenate)  (None, 35, 35, 128)  0           ['activation_2448[0][0]',        
                                                                  'activation_2450[0][0]',        
                                                                  'activation_2453[0][0]']        
                                                                                                  
 block35_1_conv (Conv2D)        (None, 35, 35, 320)  41280       ['block35_1_mixed[0][0]']        
                                                                                                  
 custom_scale_layer_480 (Custom  (None, 35, 35, 320)  0          ['mixed_5b[0][0]',               
 ScaleLayer)                                                      'block35_1_conv[0][0]']         
                                                                                                  
 block35_1

 batch_normalization_2461 (Batc  (None, 35, 35, 32)  96          ['conv2d_2461[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 batch_normalization_2464 (Batc  (None, 35, 35, 48)  144         ['conv2d_2464[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2461 (Activation)   (None, 35, 35, 32)   0           ['batch_normalization_2461[0][0]'
                                                                 ]                                
                                                                                                  
 activation_2464 (Activation)   (None, 35, 35, 48)   0           ['batch_normalization_2464[0][0]'
          

 batch_normalization_2471 (Batc  (None, 35, 35, 64)  192         ['conv2d_2471[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2466 (Activation)   (None, 35, 35, 32)   0           ['batch_normalization_2466[0][0]'
                                                                 ]                                
                                                                                                  
 activation_2468 (Activation)   (None, 35, 35, 32)   0           ['batch_normalization_2468[0][0]'
                                                                 ]                                
                                                                                                  
 activation_2471 (Activation)   (None, 35, 35, 64)   0           ['batch_normalization_2471[0][0]'
          

...

                                                                                                  
 batch_normalization_2605 (Batc  (None, 8, 8, 224)   672         ['conv2d_2605[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2605 (Activation)   (None, 8, 8, 224)    0           ['batch_normalization_2605[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_2603 (Conv2D)           (None, 8, 8, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_2606 (Conv2D)           (None, 8, 8, 256)    172032      ['activation_2605[0][0]']        
          

 conv2d_2613 (Conv2D)           (None, 8, 8, 224)    129024      ['activation_2612[0][0]']        
                                                                                                  
 batch_normalization_2613 (Batc  (None, 8, 8, 224)   672         ['conv2d_2613[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2613 (Activation)   (None, 8, 8, 224)    0           ['batch_normalization_2613[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_2611 (Conv2D)           (None, 8, 8, 192)    399360      ['block8_3_ac[0][0]']            
                                                                                                  
 conv2d_26

                                                                                                  
 conv2d_2621 (Conv2D)           (None, 8, 8, 224)    129024      ['activation_2620[0][0]']        
                                                                                                  
 batch_normalization_2621 (Batc  (None, 8, 8, 224)   672         ['conv2d_2621[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2621 (Activation)   (None, 8, 8, 224)    0           ['batch_normalization_2621[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_2619 (Conv2D)           (None, 8, 8, 192)    399360      ['block8_5_ac[0][0]']            
          

                                                                 ]                                
                                                                                                  
 conv2d_2629 (Conv2D)           (None, 8, 8, 224)    129024      ['activation_2628[0][0]']        
                                                                                                  
 batch_normalization_2629 (Batc  (None, 8, 8, 224)   672         ['conv2d_2629[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2629 (Activation)   (None, 8, 8, 224)    0           ['batch_normalization_2629[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_26

 activation_2636 (Activation)   (None, 8, 8, 192)    0           ['batch_normalization_2636[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_2637 (Conv2D)           (None, 8, 8, 224)    129024      ['activation_2636[0][0]']        
                                                                                                  
 batch_normalization_2637 (Batc  (None, 8, 8, 224)   672         ['conv2d_2637[0][0]']            
 hNormalization)                                                                                  
                                                                                                  
 activation_2637 (Activation)   (None, 8, 8, 224)    0           ['batch_normalization_2637[0][0]'
                                                                 ]                                
          

In [48]:
f1 = tfa.metrics.F1Score(num_classes=64, average='macro')

callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)

new_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-4), 
              metrics= [f1])


new_model.fit(train_data, epochs=100, callbacks=callbacks)

Epoch 1/100


2023-05-13 13:29:59.803922: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - ETA: 0s - loss: 0.6086 - f1_score: 0.0011    WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 48s 5s/step - loss: 0.6086 - f1_score: 0.0011
Epoch 2/100
4/4 [==============================] - ETA: 0s - loss: 0.4298 - f1_score: 0.0124WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 21s 5s/step - loss: 0.4298 - f1_score: 0.0124
Epoch 3/100
4/4 [==============================] - ETA: 0s - loss: 0.3021 - f1_score: 5.1230e-04WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


...

4/4 [==============================] - 21s 5s/step - loss: 9.9074e-05 - f1_score: 1.0000
Epoch 97/100
4/4 [==============================] - ETA: 0s - loss: 9.6571e-05 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 21s 5s/step - loss: 9.6571e-05 - f1_score: 1.0000
Epoch 98/100
4/4 [==============================] - ETA: 0s - loss: 9.4967e-05 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 27s 6s/step - loss: 9.4967e-05 - f1_score: 1.0000
Epoch 99/100
4/4 [==============================] - ETA: 0s - loss: 9.2024e-05 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 21s 5s/step - loss: 9.2024e-05 - f1_score: 1.0000
Epoch 100/100
4/4 [==============================] - ETA: 0s - loss: 9.1273e-05 - f1_score: 1.0000WARNING:tensorflow:Early stopping conditioned on metric `F1Score(name=f1_score,dtype=float32,num_classes=64,average=macro,threshold=None)` which is not available. Available metrics are: loss,f1_score


4/4 [==============================] - 21s 5s/step - loss: 9.1273e-05 - f1_score: 1.0000


In [49]:
new_model.save('saved_model')

INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets
